In [ ]:
import openeo

# --- Connexió al "Backend"
backend = "openeo.dataspace.copernicus.eu"
conn = openeo.connect(backend).authenticate_oidc()

# --- Dades d'entrada
satelit = "SENTINEL2_L2A"
spatial_extent = {
    "west": 3.123732,
    "east": 3.333247,
    "south": 42.123212,
    "north": 42.255935,
}
temporal_extent = ["2023-01-01", "2024-01-01"]  # Les dates estàn en notació: aaaa-mm-dd 
bands = ["B04", "B05"]  # B04 (665 nm) i B05 (705 nm) per al càlcul de clorofila
max_cloud_coverage = 0

directori_dades = "output/clorofila_mapes_golf_de_roses/"
fitxer_animacio = "output/Animacions/animacio_clorofila_golf_de_roses.gif"
titol_histograma = "Concentració d'algues: Golf de Roses"
titol_grafica_evolucio = "Evolució temporal de la concentració d'algues"

# --- Càrrega de dades del satèl·lit
datacube = conn.load_collection(
    satelit,
    spatial_extent=spatial_extent,
    temporal_extent=temporal_extent,
    bands=bands,
    max_cloud_cover=max_cloud_coverage,
)

# Normalitzem les dades (se suposa que estan 0-10.000)
datacube = datacube.apply(lambda x: x / 10000)

canal_705nm = datacube.band("B05")  # Banda 705nm
canal_665nm = datacube.band("B04")  # Banda 665nm

# Càlcul de l'índex de Clorofila-A (CI = B05 / B04)
clorofila = canal_705nm / canal_665nm

# Guardem el resultat
result = clorofila.save_result("GTiff") 

job = result.create_job()
job.start_and_wait()
job.get_results().download_files(directori_dades)

import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt

fitxers = sorted([f for f in os.listdir(directori_dades) if f.endswith('.tif')])

day_means_dict = {}

# --- Extraiem les dades dels fitxers, calculem les seves mitjanes diaries i les guardem en un diccionari
for fitxer in fitxers:
    # Extraiem la data i la mitjana diaria
    data = fitxer.split("_")[1].replace("Z.tif", "")
    with rasterio.open(directori_dades+fitxer) as dades_fitxer:
        dades = dades_fitxer.read(1)  
        mitja = np.nanmean(dades)
        day_means_dict.update({data: mitja})
        
# --- Creem l'histograma
dates = list(day_means_dict.keys())
mitjes = list(day_means_dict.values())

# Histograma de concentració d'algues
plt.figure(figsize=(10, 5))
plt.bar(dates, mitjes)
plt.title(titol_histograma)
plt.xticks(ticks=range(0, len(dates), 15), labels=[dates[i] for i in range(0, len(dates), 15)], rotation=45, ha="right")
plt.xlabel('Dates')
plt.ylabel('Mitjanes')

# --- Creem la gràfica de línia per a l'evolució temporal
plt.figure(figsize=(10, 5))
plt.plot(dates, mitjes, marker='o', linestyle='-', color='b')  # Gràfica de línia amb punts marcats
plt.title(titol_grafica_evolucio)
plt.xticks(ticks=range(0, len(dates), 15), labels=[dates[i] for i in range(0, len(dates), 15)], rotation=45, ha="right")
plt.xlabel('Dates')
plt.ylabel('Concentració mitjana de clorofila (algues)')
plt.grid(True)
plt.tight_layout()

plt.show()

from libtiff import TIFF
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os

# Fem una llista amb tots els noms dels fitxers del directori d'imatges
image_files = sorted([f for f in os.listdir(directori_dades) if f.endswith('.tif')])

def load_image(file_path):
    tif = TIFF.open(directori_dades+file_path) 
    return tif.read_image()

# Create figure and axes
fig, ax = plt.subplots()

def update(frame):
    """Update the frame for the animation."""   
    
    rgb_image = load_image(image_files[frame])
    ax.clear()
    ax.imshow(rgb_image)
    ax.set_title(f"Data: {dates[frame]}")
    ax.axis('off')
    
ani = animation.FuncAnimation(fig, update, frames=len(image_files), repeat=True, interval=200)

# Save the animation as a gif or mp4
ani.save(fitxer_animacio, writer='ffmpeg', fps=2)

plt.show()


Authenticated using refresh token.
0:00:00 Job 'j-24091574047f450c995f115c0cc66ae8': send 'start'
